In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

#read excel file into pandas dataframe
df=pd.read_excel('h_19&20.xlsx',engine='openpyxl')



In [2]:
#Setting Predictors
X=df.drop(columns=['Datetime','RH(%)']).values

#Setting Target
Y=df['RH(%)'].values

#Splitting dataset into Train and Test
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 42)
print(X_test)



[[  11   19 2020   20    0    0]
 [   6   20 2020   20    0    0]
 [   8   19 2019   21    0    0]
 ...
 [  12   14 2019    8    0    0]
 [   6   27 2019   21    0    0]
 [  10    1 2020    1    0    0]]


In [3]:

# Building  Random Forest Classifier
from sklearn.ensemble import RandomForestRegressor
rfr_hum = RandomForestRegressor(random_state =42 )
rfr_hum.fit(X_train, Y_train)


RandomForestRegressor(random_state=42)

In [4]:

# Evaluating on Training set
rfr_pred_train = rfr_hum.predict(X_train)
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
print(np.sqrt(mean_squared_error(Y_train,rfr_pred_train)))
print(r2_score(Y_train,rfr_pred_train))


# Evaluating on Test set
rfr_pred_test = rfr_hum.predict(X_test)
print(np.sqrt(mean_squared_error(Y_test,rfr_pred_test)))
print(r2_score(Y_test,rfr_pred_test))



1.3925489779084708
0.9944086194643275
3.5515513383828563
0.9636627031225364


In [5]:
#Saving to S3
import tempfile
import joblib
import boto3
s3 = boto3.resource('s3')
bucket='digitwin-airqualitydata'
key='EPA_data/EPA_humid_regress.sav'

# WRITE
with tempfile.TemporaryFile() as fp:
    joblib.dump(rfr_hum, fp)
    fp.seek(0)
    s3.Bucket(bucket).put_object(Key= key, Body=fp.read())